# First steps: Improvement
1. Model need to be cleaner lets build a sequential model
2. Lets augment artificially the images, making the distribution of data more equal

## On this part i am going to be constructing the path column
* I did a step by step frame loop, that constructed 3 sort of columns and fused then on a later ocasion
* If anyone has an idea on how to improve it(takes a while), just comment here
* The path will come in handy when we reach the data aug part, and analysis

In [ ]:
import pandas as pd
import os
import shutil

pd.set_option('display.max_colwidth', None)

train_csv = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')

* Constructin the paths

In [ ]:
os.mkdir('images/')
for i in range(100):
    i += 1
    os.mkdir(f'./images/{i}')

In [ ]:
Path = []
IMG_DIR = '../input/petfinder-pawpularity-score/train'
for i in train_csv['Id']:
    Path.append(f'{IMG_DIR}/{i}.jpg')
train_csv['Cur_Path'] = Path 
train_csv.head()

In [ ]:
Path = []
IMG_DIR = './images'
for i in train_csv['Pawpularity']:
    Path.append(f'{IMG_DIR}/{i}/')
train_csv['Wan_Path'] = Path
train_csv.head()

In [ ]:
for i,j in zip(train_csv['Cur_Path'],train_csv['Wan_Path']):
    shutil.copy2(i,j)

In [ ]:
Path = []
for i,j in zip(train_csv['Wan_Path'],train_csv['Id']):
    Path.append(f'{i}{j}.jpg')
train_csv['New_Path'] = Path

* Droping unecessary paths

In [ ]:
train_csv.drop(['Cur_Path','Wan_Path'],axis=1,inplace=True)

* Checking image size of a few pics too see how many pixels i can  pass through

In [ ]:
import PIL 

image = PIL.Image.open("./images/34/07305d3af8b61f21a0ff5e42f2101f9d.jpg")
width, height = image.size
width, height

* Taking a good luck at the distribution of values also getting rid of metadata wont be necessary

In [ ]:
train_images = train_csv[['Id','New_Path' ,'Pawpularity']]
train_images['Pawpularity'].value_counts()
#Checking the distributions of iamges

* Build up the train_df, and test_df

In [ ]:
from sklearn.model_selection import train_test_split
train_df,test_df = train_test_split(train_images,train_size=0.75,shuffle=True,random_state = 1)

* Sample to train quickly, doesnt contain all data

In [ ]:
image_sample = train_images.sample(5000, random_state = 1).reset_index(drop=True)
image_sample # For quickie test outs training

# Loading images

In [ ]:
import tensorflow as tf

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split = 0.2,
    brightness_range = (0.1,1.0),
)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    brightness_range = (0.1,1.0)
)

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'New_Path',
    y_col = 'Pawpularity',
    target_size =(120,120),
    color_mode='rgb',
    class_mode = 'raw',
    batch_size = 64,
    shuffle= True,
    seed=42,
    subset= 'training',
)


validation_images = train_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'New_Path',
    y_col = 'Pawpularity',
    target_size =(120,120),
    color_mode='rgb',
    class_mode = 'raw',
    batch_size=64,
    shuffle= True,
    seed=42,
    subset= 'validation',
)

test_images = train_generator.flow_from_dataframe(
    dataframe = test_df,
    x_col = 'New_Path',
    y_col = 'Pawpularity',
    target_size =(120,120),
    color_mode='rgb',
    class_mode = 'raw',
)

# Model (needs improvement)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential([
    layers.experimental.preprocessing.RandomFlip(mode='horizontal'),
    layers.experimental.preprocessing.RandomContrast(factor=0.1)
    ])
model = keras.Sequential(
    [data_augmentation,
     keras.layers.Conv2D(filters=16,padding='same',kernel_size=(3,3),activation='relu',input_shape=(120,120,3)),
     keras.layers.BatchNormalization(),
     keras.layers.MaxPool2D(strides=(2,2)),
     keras.layers.Conv2D(filters=32,padding='same',kernel_size=(3,3),activation='relu',input_shape=(480,480,3)),
     keras.layers.MaxPool2D(strides=(2,2)),
     keras.layers.Conv2D(filters=32,padding='same',kernel_size=(3,3),activation='relu',input_shape=(480,480,3)),
     keras.layers.MaxPool2D(strides=(2,2)),
     keras.layers.GlobalAveragePooling2D(),
     keras.layers.Dense(512,activation = 'relu'),
     keras.layers.Dense(1024,activation = 'relu'),
     keras.layers.Dense(1,activation='linear')
    ]
)
#     inputs=tf.keras.Input(shape=(240,240,3))
#     x = tf.keras.layers.Conv2D(filters=32,kernel_size=(2,2),activation='relu')(inputs)
#     x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3))(x)
#     x = tf.keras.layers.Conv2D(filters=64,kernel_size=(2,2),activation='relu')(inputs)
#     x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3))(x)
#     x = tf.keras.layers.Conv2D(filters=128,kernel_size=(2,2),activation='relu')(inputs)
#     x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3))(x)
#     x = tf.keras.layers.GlobalAveragePooling2D()(x)
#     x = tf.keras.layers.Dense(256,activation='relu')(x)
#     x = tf.keras.layers.Dense(256,activation='relu')(x)
#     outputs = tf.keras.layers.Dense(1,activation='linear')(x)
#     model = tf.keras.Model(inputs=inputs,outputs=outputs)
    
model.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss='mse')

history = model.fit(
                    train_images,
                    validation_data = validation_images,
                    epochs=100,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)])

# Seeing the end result

In [ ]:
import numpy as np
from sklearn.metrics import r2_score

predicted_paws = np.squeeze(model.predict(test_images))
true_paw=test_images.labels

rmse=np.sqrt(model.evaluate(test_images,verbose=0))
print(f'RMSE {rmse:.5f}')
r2=r2_score(true_paw,predicted_paws)
print(f'R2 {r2:.5f}')

In [ ]:
np.mean(true_paw)

### Making the CSV file

In [ ]:
Submission_df = pd.Series([i for i in test_df['Id']]).to_frame(name='Id')
# It is okay it has random state is always the same

In [ ]:
dirty_predicted_paw = np.squeeze(model.predict(test_images))
Submission_df['Pawpularity']=pd.Series([f'{i:.2f}' for i in dirty_predicted_paw]).astype('float32')
# I know right quite hard to read, i am not even sorry round function is bugged as hell

In [ ]:
Submission_df.dtypes

In [ ]:
os.mkdir
Submission_df.to_csv('./submission.csv',index=False,encoding='utf-8')
#Fix format later